In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
     
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data preprocessing

In [2]:
# Load the data
#clicks_df = pd.read_csv('/kaggle/input/recsys-vacancies-seekers/subsample_link_seekers_vacancies.csv')
seekers_df = pd.read_csv('/kaggle/input/recsys-vacancies-seekers/subsample_seekers.csv')
vacancies_df = pd.read_csv('/kaggle/input/recsys-vacancies-seekers/subsample_vacancies.csv')

In [3]:
clicks_df

,VacancyIdHashed,SeekerIdHashed
0,4455325407663931,8365881490387660854
1,8178571888231674,7570977539248865846
2,9960840192558450,2701394621277898844
3,10715889754036128,5744794685959173407
4,12172124159353673,3799051775082640729
...,...,...
120790,9213542662014160345,4647815836829893024
120791,9213628176729723795,221601999793491061
120792,9214490111061193644,1421265644905800898
120793,9215629624538255757,3491509468637873733


In [4]:
seekers_df

,SeekerIdHashed,Gender,City,RegistrationTime,SferaDeyatelnosti,AllContactsForJob
0,8239136462566553770,male,Санкт-Петербург,2011-11-22 23:09:22.592276,Административная работа,1
1,8239136462566553770,male,Санкт-Петербург,2011-11-22 23:09:22.592276,"Без опыта, студенты",4
2,8239136462566553770,male,Санкт-Петербург,2011-11-22 23:09:22.592276,"Искусство, развлечения",1
3,8239136462566553770,male,Санкт-Петербург,2011-11-22 23:09:22.592276,Курьерская доставка,1
4,8239136462566553770,male,Санкт-Петербург,2011-11-22 23:09:22.592276,"Охрана, безопасность",10
...,...,...,...,...,...,...
216344,2265241279192812153,male,Санкт-Петербург,2016-10-18 23:04:00.056322,"Без опыта, студенты",1
216345,2265241279192812153,male,Санкт-Петербург,2016-10-18 23:04:00.056322,"Производство, сырьё, с/х",3
216346,2265241279192812153,male,Санкт-Петербург,2016-10-18 23:04:00.056322,Строительство,2
216347,2265241279192812153,male,Санкт-Петербург,2016-10-18 23:04:00.056322,"Транспорт, логистика",1


In [6]:
df = clicks_df.set_index('SeekerIdHashed').join(seekers_df.set_index('SeekerIdHashed'), how='inner')
df.reset_index(inplace=True)

In [7]:
df

,SeekerIdHashed,VacancyIdHashed,Gender,City,RegistrationTime,SferaDeyatelnosti,AllContactsForJob
0,92777930943472,555109274142421377,NaN,Нижний Новгород,2011-11-08 05:47:21.114599,Административная работа,1
1,92777930943472,555109274142421377,NaN,Нижний Новгород,2011-11-08 05:47:21.114599,"Без опыта, студенты",3
2,92777930943472,555109274142421377,NaN,Нижний Новгород,2011-11-08 05:47:21.114599,Домашний персонал,1
3,92777930943472,555109274142421377,NaN,Нижний Новгород,2011-11-08 05:47:21.114599,"Образование, наука",1
4,92777930943472,555109274142421377,NaN,Нижний Новгород,2011-11-08 05:47:21.114599,Продажи,1
...,...,...,...,...,...,...,...
819681,9222041705582243930,8336958509020282500,female,Белгород,2015-03-19 17:13:50.039816,Домашний персонал,1
819682,9222041705582243930,8336958509020282500,female,Белгород,2015-03-19 17:13:50.039816,"Медицина, фармацевтика",1
819683,9222041705582243930,8336958509020282500,female,Белгород,2015-03-19 17:13:50.039816,"Образование, наука",8
819684,9222041705582243930,8336958509020282500,female,Белгород,2015-03-19 17:13:50.039816,Продажи,3


In [8]:
vac_groupby_city_field = vacancies_df.groupby(['City', 'SferaDeyatelnosti']).agg(lambda x : x)

In [9]:
vac_groupby_city_field

VacancyIdHashed  \
City        SferaDeyatelnosti                                                             
Абадзехская Бухгалтерия, финансы                                          1438045750209   
Абакан      IT, интернет, телеком     [802187250089, 1429451752068, 1402635500278, 9...   
            Автомобильный бизнес      [1435936503488, 1435028000086, 1234923751397, ...   
            Административная работа   [1399285001430, 1305171000285, 1440061002300, ...   
            Банки, инвестиции                                             1440001000838   
...                                                                                 ...   
Ясногорск   Производство, сырьё, с/х                      [850727250263, 1438061751937]   
            Транспорт, логистика          [1365877750039, 1440558253515, 1433199500980]   
Яхрома      Без опыта, студенты                          [1441206250887, 1438056251574]   
            Продажи                                      [1422609500611, 1435188750014]   
            Производство, сырьё, с/х                     [1439658750388, 1434796501232]   

                                                                           GrafikRaboty  \
City        SferaDeyatelnosti                                                             
Абадзехская Бухгалтерия, финансы                                            Полный день   
Абакан      IT, интернет, телеком     [Полный день, Свободный график, Сменный график...   
            Автомобильный бизнес      [Сменный график, Полный день, Свободный график...   
            Административная работа   [Сменный график, Полный день, Сменный график, ...   
            Банки, инвестиции                                               Полный день   
...                                                                                 ...   
Ясногорск   Производство, сырьё, с/х                      [Полный день, Вахтовый метод]   
            Транспорт, логистика             [Сменный график, Полный день, Полный день]   
Яхрома      Без опыта, студенты                        [Сменный график, Вахтовый метод]   
            Продажи                                     [Свободный график, Полный день]   
            Производство, сырьё, с/х                    [Свободный график, Полный день]   

                                                                             Professiya  \
City        SferaDeyatelnosti                                                             
Абадзехская Бухгалтерия, финансы                                              Бухгалтер   
Абакан      IT, интернет, телеком     [Программист, Менеджер по рекламе, Администрат...   
            Автомобильный бизнес      [Автослесарь, Водитель автобуса, Ремонтник, Та...   
            Административная работа   [Менеджер, Администратор, Администратор, Админ...   
            Банки, инвестиции                                      Кредитный специалист   
...                                                                                 ...   
Ясногорск   Производство, сырьё, с/х                          [Оператор, Не определено]   
            Транспорт, логистика      [Водитель самосвала, Автослесарь, Водитель авт...   
Яхрома      Без опыта, студенты                                   [Упаковщик, Оператор]   
            Продажи                              [Не определено, Агент по недвижимости]   
            Производство, сырьё, с/х                                [Инженер, Менеджер]   

                                                                                  Price  \
City        SferaDeyatelnosti                                                             
Абадзехская Бухгалтерия, финансы                                                52500.0   
Абакан      IT, интернет, телеком                      [30000.0, 3850.0, -1.0, 30000.0]   
            Автомобильный бизнес      [75000.0, 40500.0, 40000.0, 750.0, 50000.0, 60...   
            Административная работа   [18000.0, 27500.0, 32000.0,

In [45]:
np.random.choice(vac_groupby_city_field.loc['Абакан', 'Административная работа'][0])

1399285001430

In [80]:
vac_groupby_city_field.index

MultiIndex([('Абадзехская',     'Бухгалтерия, финансы'),
            (     'Абакан',    'IT, интернет, телеком'),
            (     'Абакан',     'Автомобильный бизнес'),
            (     'Абакан',  'Административная работа'),
            (     'Абакан',        'Банки, инвестиции'),
            (     'Абакан',      'Без опыта, студенты'),
            (     'Абакан',     'Бухгалтерия, финансы'),
            (     'Абакан',           'Госслужба, НКО'),
            (     'Абакан',        'Домашний персонал'),
            (     'Абакан',        'ЖКХ, эксплуатация'),
            ...
            (     'Ярцево', 'Производство, сырьё, с/х'),
            (     'Ярцево',            'Строительство'),
            (     'Ярцево',        'Туризм, рестораны'),
            (  'Ясногорск',     'Автомобильный бизнес'),
            (  'Ясногорск',                  'Продажи'),
            (  'Ясногорск', 'Производство, сырьё, с/х'),
            (  'Ясногорск',     'Транспорт, логистика'),
            (  

In [10]:
vac_groupby_city_field.index

MultiIndex([('Абадзехская',     'Бухгалтерия, финансы'),
            (     'Абакан',    'IT, интернет, телеком'),
            (     'Абакан',     'Автомобильный бизнес'),
            (     'Абакан',  'Административная работа'),
            (     'Абакан',        'Банки, инвестиции'),
            (     'Абакан',      'Без опыта, студенты'),
            (     'Абакан',     'Бухгалтерия, финансы'),
            (     'Абакан',           'Госслужба, НКО'),
            (     'Абакан',        'Домашний персонал'),
            (     'Абакан',        'ЖКХ, эксплуатация'),
            ...
            (     'Ярцево', 'Производство, сырьё, с/х'),
            (     'Ярцево',            'Строительство'),
            (     'Ярцево',        'Туризм, рестораны'),
            (  'Ясногорск',     'Автомобильный бизнес'),
            (  'Ясногорск',                  'Продажи'),
            (  'Ясногорск', 'Производство, сырьё, с/х'),
            (  'Ясногорск',     'Транспорт, логистика'),
            (  

In [12]:
vacancies_df_np = vacancies_df.to_numpy()

In [15]:
vacancies_df_np

array([[1436777253438, 'Новороссийск', 'Продажи', ..., 31, 2,
        'Требуется менеджер по продажам электротоваров. Ответственный, инициативный, исполнительный, доброжелательный.'],
       [1435550257122, 'Москва', 'Охрана, безопасность', ..., 669, 35,
        'Охранная организация «Невский Контроль» проводит набор сотрудников охраны!\n\nДолжностные обязанности:\n- контроль пропускного режима;\n- контроль видеонаблюдения;\n- недопущение на объект посторонних лиц;\n- ведение сопутствующей документации;\n- поддержание чистоты на КПП и прилегающей к ней территории;\n\nУсловия:\n- заработная плата 2 раза в месяц, авансирование\nСмена лицензированного сотрудника от 2700р до 3000р \nРАССМАТРИВАЕМ СОИСКАТЕЛЕЙ БЕЗ УДОСТОВЕРЕНИЯ ОХРАНННИКА(лицензия), но с желанием лицензироваться, помогаем в получении удостоверения! \n- выдаём форменную одежду\n- РФ'],
       [1437117755975, 'Челябинск', 'Продажи', ..., 327, 11,
        'Описание работодателя: Наша компания ООО «АВТОСПЕЦТЕХНИКА ГРУПП» занимае

In [19]:
def change_vacid(row, vac_groupby_city_field=vac_groupby_city_field):
    curr_City = row.City
    curr_SferaDeyatelnosti = row.SferaDeyatelnosti
    #print((curr_City, curr_SferaDeyatelnosti))
    if (curr_City, curr_SferaDeyatelnosti) in vac_groupby_city_field.index:
        #print('im here')
        random_id = np.random.choice(vac_groupby_city_field.loc[curr_City, curr_SferaDeyatelnosti][0])
        #print('im here')
    else:
        #if curr_SferaDeyatelnosti in vac_groupby_city_field.loc[curr_City].index:
        (rand_town, rand_field) = np.random.choice(vac_groupby_city_field.index)
        random_id = np.random.choice(vac_groupby_city_field.loc[rand_town, rand_field][0])
        #else:
        #    deyat_list = vac_groupby_city_field.loc[curr_City].index
         #   deyat = np.random.choice(deyat_list)
          #  random_id = np.random.choice(vac_groupby_city_field.loc[curr_City, deyat][0]) 
        
    row.VacancyIdHashed = random_id
    return row

In [20]:
df1 = df.apply(change_vacid, axis=1)

In [23]:
df1.to_csv('/kaggle/working/df_link_seek_vac.csv')

In [31]:
df2 = df1.drop(['City', 'SferaDeyatelnosti'], axis=1)

In [32]:
df_full = df2.set_index('VacancyIdHashed').join(vacancies_df.set_index('VacancyIdHashed'), how='inner')
df_full.reset_index(inplace=True)

In [34]:
df_full.to_csv('/kaggle/working/df_full.csv')

# EDA

In [106]:
df_full.columns

Index(['Unnamed: 0', 'VacancyIdHashed', 'SeekerIdHashed', 'Gender',
       'RegistrationTime', 'AllContactsForJob', 'City', 'SferaDeyatelnosti',
       'GrafikRaboty', 'Professiya', 'Price', 'EmployerRating', 'Views',
       'Contacts', 'Desrciption'],
      dtype='object')

In [125]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 758085 entries, 0 to 758084
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   VacancyIdHashed    758085 non-null  int64  
 1   SeekerIdHashed     758085 non-null  int64  
 2   Gender             758085 non-null  object 
 3   RegistrationTime   758085 non-null  object 
 4   AllContactsForJob  758085 non-null  int64  
 5   City               758085 non-null  object 
 6   SferaDeyatelnosti  758085 non-null  object 
 7   GrafikRaboty       758085 non-null  object 
 8   Professiya         758085 non-null  object 
 9   Price              758085 non-null  float64
 10  EmployerRating     758085 non-null  float64
 11  Views              758085 non-null  int64  
 12  Contacts           758085 non-null  int64  
 13  Desrciption        758085 non-null  object 
dtypes: float64(2), int64(5), object(7)
memory usage: 81.0+ MB


In [107]:
df_full.drop(['Unnamed: 0'], axis=1, inplace=True)

In [99]:
df_full.Gender = df_full.Gender.fillna(method='ffill')

In [128]:
df_full.sample(10)

,VacancyIdHashed,SeekerIdHashed,Gender,RegistrationTime,AllContactsForJob,City,SferaDeyatelnosti,GrafikRaboty,Professiya,Price,EmployerRating,Views,Contacts,Desrciption
513276,1435644006189,7009617160251605203,female,2013-04-16 17:44:45.874591,1,Екатеринбург,Управление персоналом,Полный день,Менеджер,75000.0,4.0,54,8,Описание работодателя: \nТребования:\nОбязанно...
653935,1439331002578,8847608850332358348,male,2014-07-10 15:02:30.115966,13,Рязань,Курьерская доставка,Полный день,Курьер,96000.0,2.0,18,2,Нажимайте Откликнуться и мы с Вами свяжемся. З...
64184,940526500826,3259640195562773680,female,2014-04-15 14:32:19.451238,2,Чита,Продажи,Полный день,Водитель,54500.0,5.0,635,37,Требования: • наличие водительского удостовере...
529602,1436182752863,7177748790399456700,male,2014-10-14 17:44:10.748701,4,Москва,Административная работа,Полный день,Водитель,60000.0,5.0,340,33,"Обращайтесь, если вы живете не более чем в 30 ..."
683697,1440082751734,4915669658886304723,male,2015-10-25 12:25:14.636076,1,Симферополь,Продажи,Полный день,Грузчик,40000.0,4.0,181,14,В дистрибьюторской компании (бытовая химия) от...
182302,1284027500049,1383371132341560402,male,2012-11-27 08:31:50.059348,1,Омск,"ЖКХ, эксплуатация",Сменный график,Не определено,7000.0,4.0,27,2,Описание работодателя: требуется уборщица в ТЦ...
248177,1354089500369,5120054956056818689,female,2016-04-20 19:38:04.224705,4,Орехово-Зуево,"Искусство, развлечения",Сменный график,Повар,49000.0,4.0,28,2,В кафе бар требуется на постоянную работу пова...
329332,1410010250232,1068093409093977624,female,2014-07-31 22:30:23.722900,20,Ростов,"Производство, сырьё, с/х",Полный день,Швея,55000.0,4.0,27,3,"Требования: Аккуратность, внимательность, обуч..."
679461,1440062252072,7704185867421202923,female,2016-01-26 10:44:03.296994,1,Оренбург,Продажи,Полный день,Продавец-консультант,1200.0,5.0,667,31,Уличная торговля 7-8-9 числа торговля вербой 1000
552116,1437123505382,8263277334380980056,female,2013-01-09 20:42:41.972775,78,Калининград,Продажи,Сменный график,Продавец-консультант,27000.0,5.0,684,44,"В ТС ""Дельта-Дисконт"" торговля бытовой химией ..."


In [124]:
df_full['EmployerRating'].describe()

count    758085.000000
mean          4.074932
std           0.982098
min           0.000000
25%           4.000000
50%           4.000000
75%           5.000000
max           5.000000
Name: EmployerRating, dtype: float64

In [81]:
df_full.Professiya.value_counts()

Не определено           65659
Водитель                52995
Продавец-консультант    35699
Курьер                  26234
Администратор           26056
                        ...  
Облицовщик                 10
Дистрибьютор                6
Пилот                       5
Стекольщик                  4
Агент рекламный             2
Name: Professiya, Length: 241, dtype: int64

In [86]:
# def change_undefined_to_sth(a):
#     if a == 'Не определено':
#         return 'любыеРаботыВыполняюВнатуре'
        

In [118]:
df_full.Professiya = df_full.Professiya.fillna('Не определено')

In [123]:
df_full.EmployerRating = df_full.EmployerRating.fillna(4.0)

In [119]:
df_full.Professiya.value_counts()

Не определено           65779
Водитель                52995
Продавец-консультант    35699
Курьер                  26234
Администратор           26056
                        ...  
Облицовщик                 10
Дистрибьютор                6
Пилот                       5
Стекольщик                  4
Агент рекламный             2
Name: Professiya, Length: 241, dtype: int64

In [127]:
df_full.to_csv('/kaggle/working/df_full.csv', index=False)

# start here

In [3]:
data = pd.read_csv('/kaggle/input/recsys-vacancies-seekers/df_full.csv')

In [15]:
data_useful

,VacancyIdHashed,SeekerIdHashed,RegistrationTime,AllContactsForJob,City,Professiya,Price,EmployerRating,Views,Contacts,Desrciption,"SferaDeyatelnosti_IT, интернет, телеком",SferaDeyatelnosti_Автомобильный бизнес,SferaDeyatelnosti_Административная работа,"SferaDeyatelnosti_Банки, инвестиции","SferaDeyatelnosti_Без опыта, студенты","SferaDeyatelnosti_Бухгалтерия, финансы",SferaDeyatelnosti_Высший менеджмент,"SferaDeyatelnosti_Госслужба, НКО",SferaDeyatelnosti_Домашний персонал,"SferaDeyatelnosti_ЖКХ, эксплуатация","SferaDeyatelnosti_Искусство, развлечения",SferaDeyatelnosti_Консультирование,SferaDeyatelnosti_Курьерская доставка,"SferaDeyatelnosti_Маркетинг, реклама, PR","SferaDeyatelnosti_Медицина, фармацевтика","SferaDeyatelnosti_Образование, наука","SferaDeyatelnosti_Охрана, безопасность",SferaDeyatelnosti_Продажи,"SferaDeyatelnosti_Производство, сырьё, с/х",SferaDeyatelnosti_Страхование,SferaDeyatelnosti_Строительство,SferaDeyatelnosti_Такси,"SferaDeyatelnosti_Транспорт, логистика","SferaDeyatelnosti_Туризм, рестораны",SferaDeyatelnosti_Управление персоналом,"SferaDeyatelnosti_Фитнес, салоны красоты",SferaDeyatelnosti_Юриспруденция,GrafikRaboty_Вахтовый метод,GrafikRaboty_Неполный день,GrafikRaboty_Полный день,GrafikRaboty_Свободный график,GrafikRaboty_Сменный график,GrafikRaboty_Удалённая работа,Gender_company,Gender_couple,Gender_female,Gender_male,Gender_male/female
0,935629,510088507149569965,2015-12-07 18:41:55.775048,4,Сочи,Не определено,45000.0,5.0,321,9,ЧИТАЕМ ВНИМАТЕЛЬНО Приглашаем на работу в мага...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,935629,3493010241992275817,2016-07-04 20:06:55.973613,15,Сочи,Не определено,45000.0,5.0,321,9,ЧИТАЕМ ВНИМАТЕЛЬНО Приглашаем на работу в мага...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
2,935629,4337530415788530088,2014-07-16 10:15:09.059350,1,Сочи,Не определено,45000.0,5.0,321,9,ЧИТАЕМ ВНИМАТЕЛЬНО Приглашаем на работу в мага...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,935629,4366126993136331471,2016-03-02 11:33:55.589401,2,Сочи,Не определено,45000.0,5.0,321,9,ЧИТАЕМ ВНИМАТЕЛЬНО Приглашаем на работу в мага...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,935629,6163790634367727090,2016-04-09 10:31:59.347688,2,Сочи,Не определено,45000.0,5.0,321,9,ЧИТАЕМ ВНИМАТЕЛЬНО Приглашаем на работу в мага...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758080,1441394750644,3262197496291850079,2013-11-10 16:57:06.854910,9,Москва,Дизайнер,-1.0,4.0,65,6,Ищу чертежника АвтокадРевит на долгосрочную ра...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
758081,1441394750644,3734938522353927649,2013-02-18 21:46:34.149662,2,Москва,Дизайнер,-1.0,4.0,65,6,Ищу чертежника АвтокадРевит на долгосрочную ра...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
758082,1441394750644,5024892471381985020,2011-01-12 01:01:37.803142,1,Москва,Дизайнер,-1.0,4.0,65,6,Ищу чертежника АвтокадРевит на долгосрочную ра...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
758083,1441394750644,8026820411723370631,2012-09-12 22:34:48.067673,3,Москва,Дизайнер,-1.0,4.0,65,6,Ищу чертежника АвтокадРевит на долгосрочную ра...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [4]:
cols_to_ohe = ['SferaDeyatelnosti', 'GrafikRaboty', 'Gender']
data_useful = pd.get_dummies(data, columns=cols_to_ohe)

In [6]:
import re
data_useful['Desrciption'] = data_useful['Desrciption'].apply(lambda x: re.sub(r'[^\w\sа-яА-Я]', '', x))

In [7]:
import torch
import tensorflow as tf
import tensorflow_hub as hub
from transformers import AutoTokenizer, AutoModel

# проверка доступности GPU для PyTorch и TensorFlow
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Using device:", device)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using device: cuda


In [8]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        print(e)

Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [17]:
import pandas as pd

def max_length(df, column_name='Desrciption'):
    """
    Функция для определения максимальной длины строки в колонке датафрейма
    
    Параметры:
    df (pandas.DataFrame): Датафрейм
    column_name (str): Название колонки, для которой нужно определить максимальную длину
    
    Возвращает:
    int: Максимальная длина строки в колонке датафрейма
    """
    max_len = 0
    for text in df[column_name]:
        if len(text) > max_len:
            max_len = len(text)
    return max_len

def truncate(df, length, column_name='Desrciption'):
    """
    Функция для обрезания всех значений в колонке датафрейма до заданной длины
    
    Параметры:
    df (pandas.DataFrame): Датафрейм
    column_name (str): Название колонки, которую нужно обрезать
    length (int): Длина, до которой нужно обрезать строки
    
    Возвращает:
    pandas.DataFrame: Датафрейм с обрезанными строками в колонке
    """
    df_truncated = df.copy()
    df_truncated[column_name] = df_truncated[column_name].apply(lambda x: x[:length])
    return df_truncated


In [18]:
# Определим максимальную длину строки в колонке 'Description'
max_len = max_length(data_useful)
print(f'Максимальная длина: {max_len}')

Максимальная длина: 5521


In [20]:
# Обрежем все значения в колонке 'Description' до 20 символов
data_useful_truncated = truncate(data_useful, 500)
data_useful_truncated

,VacancyIdHashed,SeekerIdHashed,RegistrationTime,AllContactsForJob,City,Professiya,Price,EmployerRating,Views,Contacts,Desrciption,"SferaDeyatelnosti_IT, интернет, телеком",SferaDeyatelnosti_Автомобильный бизнес,SferaDeyatelnosti_Административная работа,"SferaDeyatelnosti_Банки, инвестиции","SferaDeyatelnosti_Без опыта, студенты","SferaDeyatelnosti_Бухгалтерия, финансы",SferaDeyatelnosti_Высший менеджмент,"SferaDeyatelnosti_Госслужба, НКО",SferaDeyatelnosti_Домашний персонал,"SferaDeyatelnosti_ЖКХ, эксплуатация","SferaDeyatelnosti_Искусство, развлечения",SferaDeyatelnosti_Консультирование,SferaDeyatelnosti_Курьерская доставка,"SferaDeyatelnosti_Маркетинг, реклама, PR","SferaDeyatelnosti_Медицина, фармацевтика","SferaDeyatelnosti_Образование, наука","SferaDeyatelnosti_Охрана, безопасность",SferaDeyatelnosti_Продажи,"SferaDeyatelnosti_Производство, сырьё, с/х",SferaDeyatelnosti_Страхование,SferaDeyatelnosti_Строительство,SferaDeyatelnosti_Такси,"SferaDeyatelnosti_Транспорт, логистика","SferaDeyatelnosti_Туризм, рестораны",SferaDeyatelnosti_Управление персоналом,"SferaDeyatelnosti_Фитнес, салоны красоты",SferaDeyatelnosti_Юриспруденция,GrafikRaboty_Вахтовый метод,GrafikRaboty_Неполный день,GrafikRaboty_Полный день,GrafikRaboty_Свободный график,GrafikRaboty_Сменный график,GrafikRaboty_Удалённая работа,Gender_company,Gender_couple,Gender_female,Gender_male,Gender_male/female
0,935629,510088507149569965,2015-12-07 18:41:55.775048,4,Сочи,Не определено,45000.0,5.0,321,9,ЧИТАЕМ ВНИМАТЕЛЬНО Приглашаем на работу в мага...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,935629,3493010241992275817,2016-07-04 20:06:55.973613,15,Сочи,Не определено,45000.0,5.0,321,9,ЧИТАЕМ ВНИМАТЕЛЬНО Приглашаем на работу в мага...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
2,935629,4337530415788530088,2014-07-16 10:15:09.059350,1,Сочи,Не определено,45000.0,5.0,321,9,ЧИТАЕМ ВНИМАТЕЛЬНО Приглашаем на работу в мага...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,935629,4366126993136331471,2016-03-02 11:33:55.589401,2,Сочи,Не определено,45000.0,5.0,321,9,ЧИТАЕМ ВНИМАТЕЛЬНО Приглашаем на работу в мага...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,935629,6163790634367727090,2016-04-09 10:31:59.347688,2,Сочи,Не определено,45000.0,5.0,321,9,ЧИТАЕМ ВНИМАТЕЛЬНО Приглашаем на работу в мага...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758080,1441394750644,3262197496291850079,2013-11-10 16:57:06.854910,9,Москва,Дизайнер,-1.0,4.0,65,6,Ищу чертежника АвтокадРевит на долгосрочную ра...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
758081,1441394750644,3734938522353927649,2013-02-18 21:46:34.149662,2,Москва,Дизайнер,-1.0,4.0,65,6,Ищу чертежника АвтокадРевит на долгосрочную ра...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
758082,1441394750644,5024892471381985020,2011-01-12 01:01:37.803142,1,Москва,Дизайнер,-1.0,4.0,65,6,Ищу чертежника АвтокадРевит на долгосрочную ра...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
758083,1441394750644,8026820411723370631,2012-09-12 22:34:48.067673,3,Москва,Дизайнер,-1.0,4.0,65,6,Ищу чертежника АвтокадРевит на долгосрочную ра...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [21]:
# загрузка предобученной модели BERT и токенизатора
model = AutoModel.from_pretrained("bert-base-uncased").to(device)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
data_useful_truncated.iloc[:1000, :]

In [ ]:
# загрузка предобученной модели USE
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4").to(device)

# функция для получения эмбеддинга для текста с помощью BERT
def get_bert_embedding(text):
    # токенизация текста и добавление токена [CLS] в начало и [SEP] в конец
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)]).to(device)
    # получение эмбеддингов для всех токенов с помощью BERT
    with torch.no_grad():
        outputs = model(input_ids)
    # получение эмбеддинга для всего текста из эмбеддингов токенов
    embedding = torch.mean(outputs[0], dim=1).squeeze().cpu().numpy()
    return embedding

# функция для получения эмбеддинга для текста с помощью USE
# def get_use_embedding(text):
#     # получение эмбеддинга для всего текста с помощью USE
#     embedding = embed([text])[0].numpy()
#     return embedding

data_temp = data_useful_truncated.iloc[:1000, :]
# создание новых колонок "bert_embedding" и "use_embedding" со значениями эмбеддингов для каждого текста
bert_emd_col = data_useful_truncated['Desrciption'].map(lambda x: get_bert_embedding(x))
# use_emd_col = data_useful['Description'].apply(lambda x: get_use_embedding(x))

'ЧИТАЕМ ВНИМАТЕЛЬНО Приглашаем на работу в магазин  Веселая Затея в Адлере в ТРЦ Плаза торговые павильоны творческих  талантливых сотрудниц Специфика воздушные шары и всё для праздника и карнавала упаковка подарков работа с ПК основы векторной графики 1 С используем мессенджеры и соцсети Скучно не будет Работа для тех кому интересно заниматься творчеством и помогать делать праздник БЕЗ ВРЕДНЫХ предпочтений Заработная плата от 45 т руб График работы 2 через 2 Просьба звонить с 10 до 19 часов директору магазина на номер 8 928 из наших контактов правила Авито'

In [8]:
from sklearn.preprocessing import StandardScaler
pca1 = StandardScaler()
data_useful[['AllContactsForJob', 'Price', 'EmployerRating', 'Views', 'Contacts']] = pca1.fit_transform(data_useful[['AllContactsForJob', 'Price', 'EmployerRating', 'Views', 'Contacts']])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
data_useful[data_useful.SeekerIdHashed == 3493010241992275817]

,VacancyIdHashed,SeekerIdHashed,RegistrationTime,AllContactsForJob,City,Professiya,Price,EmployerRating,Views,Contacts,...,GrafikRaboty_Неполный день,GrafikRaboty_Полный день,GrafikRaboty_Свободный график,GrafikRaboty_Сменный график,GrafikRaboty_Удалённая работа,Gender_company,Gender_couple,Gender_female,Gender_male,Gender_male/female
1,935629,3493010241992275817,2016-07-04 20:06:55.973613,0.612506,Сочи,Не определено,0.000083,0.941931,0.032082,-0.256906,...,0,0,0,1,0,0,0,1,0,0
7333,407852500603,3493010241992275817,2016-07-04 20:06:55.973613,0.072881,Сочи,заправщик,-1.021713,-0.076298,-0.515491,-0.377341,...,0,0,0,1,0,0,0,1,0,0
19517,633626750335,3493010241992275817,2016-07-04 20:06:55.973613,0.612506,Сочи,Продавец-консультант,-0.283743,-4.149214,-0.394674,-0.305080,...,0,1,0,0,0,0,0,1,0,0
25269,685856000743,3493010241992275817,2016-07-04 20:06:55.973613,-0.264385,Сочи,Швея,0.476911,-0.076298,-0.375187,-0.377341,...,0,1,0,0,0,0,0,1,0,0
26785,689683773901,3493010241992275817,2016-07-04 20:06:55.973613,-0.129479,Сочи,Не определено,-0.113447,-0.076298,-0.496004,-0.377341,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732069,1440890000802,3493010241992275817,2016-07-04 20:06:55.973613,-0.331838,Сочи,Администратор,0.056849,-0.076298,0.262023,-0.016036,...,0,0,0,1,0,0,0,1,0,0
733660,1440896752518,3493010241992275817,2016-07-04 20:06:55.973613,0.072881,Сочи,Кассир,-0.113447,-3.130985,-0.416109,-0.305080,...,0,0,0,1,0,0,0,1,0,0
734923,1440909006055,3493010241992275817,2016-07-04 20:06:55.973613,-0.331838,Сочи,Администратор,0.000083,-0.076298,-0.032224,-0.256906,...,0,1,0,0,0,0,0,1,0,0
745415,1440991251053,3493010241992275817,2016-07-04 20:06:55.973613,-0.331838,Сочи,Не определено,0.340675,-0.076298,0.186026,0.128486,...,0,1,0,0,0,0,0,1,0,0


In [15]:
seekers_df[seekers_df.SeekerIdHashed == 3493010241992275817].iloc[0].City

'Сочи'

In [27]:
list(data_useful.columns)[3:4] + list(data_useful.columns)[6:10] + list(data_useful.columns)[11:]

['AllContactsForJob',
 'Price',
 'EmployerRating',
 'Views',
 'Contacts',
 'SferaDeyatelnosti_IT, интернет, телеком',
 'SferaDeyatelnosti_Автомобильный бизнес',
 'SferaDeyatelnosti_Административная работа',
 'SferaDeyatelnosti_Банки, инвестиции',
 'SferaDeyatelnosti_Без опыта, студенты',
 'SferaDeyatelnosti_Бухгалтерия, финансы',
 'SferaDeyatelnosti_Высший менеджмент',
 'SferaDeyatelnosti_Госслужба, НКО',
 'SferaDeyatelnosti_Домашний персонал',
 'SferaDeyatelnosti_ЖКХ, эксплуатация',
 'SferaDeyatelnosti_Искусство, развлечения',
 'SferaDeyatelnosti_Консультирование',
 'SferaDeyatelnosti_Курьерская доставка',
 'SferaDeyatelnosti_Маркетинг, реклама, PR',
 'SferaDeyatelnosti_Медицина, фармацевтика',
 'SferaDeyatelnosti_Образование, наука',
 'SferaDeyatelnosti_Охрана, безопасность',
 'SferaDeyatelnosti_Продажи',
 'SferaDeyatelnosti_Производство, сырьё, с/х',
 'SferaDeyatelnosti_Страхование',
 'SferaDeyatelnosti_Строительство',
 'SferaDeyatelnosti_Такси',
 'SferaDeyatelnosti_Транспорт, логис

In [23]:
list(data_useful.columns)

['VacancyIdHashed',
 'SeekerIdHashed',
 'RegistrationTime',
 'AllContactsForJob',
 'City',
 'Professiya',
 'Price',
 'EmployerRating',
 'Views',
 'Contacts',
 'Desrciption',
 'SferaDeyatelnosti_IT, интернет, телеком',
 'SferaDeyatelnosti_Автомобильный бизнес',
 'SferaDeyatelnosti_Административная работа',
 'SferaDeyatelnosti_Банки, инвестиции',
 'SferaDeyatelnosti_Без опыта, студенты',
 'SferaDeyatelnosti_Бухгалтерия, финансы',
 'SferaDeyatelnosti_Высший менеджмент',
 'SferaDeyatelnosti_Госслужба, НКО',
 'SferaDeyatelnosti_Домашний персонал',
 'SferaDeyatelnosti_ЖКХ, эксплуатация',
 'SferaDeyatelnosti_Искусство, развлечения',
 'SferaDeyatelnosti_Консультирование',
 'SferaDeyatelnosti_Курьерская доставка',
 'SferaDeyatelnosti_Маркетинг, реклама, PR',
 'SferaDeyatelnosti_Медицина, фармацевтика',
 'SferaDeyatelnosti_Образование, наука',
 'SferaDeyatelnosti_Охрана, безопасность',
 'SferaDeyatelnosti_Продажи',
 'SferaDeyatelnosti_Производство, сырьё, с/х',
 'SferaDeyatelnosti_Страхование',


In [28]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Подготовка данных для использования в алгоритме cosine distance
features = list(data_useful.columns)[3:4] + list(data_useful.columns)[6:10] + list(data_useful.columns)[11:]
# for feature in features:
#     data_useful[feature] = pd.Categorical(data_useful[feature]).codes

# Функция для нахождения топ n рекомендаций для заданного соискателя
def get_top_n_recommendations(seeker_id, n, data_useful=data_useful, seekers_df=seekers_df, vacancies_df=vacancies_df):
    approp_city = seekers_df[seekers_df.SeekerIdHashed == seeker_id].iloc[0].City
    print(approp_city)
    data_useful = data_useful[data_useful.City == approp_city]
    seeker_data = data_useful[data_useful['SeekerIdHashed'] == seeker_id]
    seeker_features = seeker_data[features].values
    vacancy_data = data_useful[data_useful['SeekerIdHashed'] != seeker_id]
    vacancy_features = vacancy_data[features].values
    similarity_scores = cosine_similarity(seeker_features, vacancy_features)
    top_n_indices = similarity_scores.argsort()[0][-n:]
    print(similarity_scores)
    top_n_recommendations = vacancy_data.iloc[top_n_indices]['VacancyIdHashed']
    return top_n_recommendations.tolist()

In [44]:
%%time
relevant_vacancies = get_top_n_recommendations(3493010241992275817, 300)

Сочи
[[ 0.69336944  0.65581294  0.66898012 ...  0.39899704  0.39899704
   0.38833321]
 [ 0.23730088  0.23109713  0.23340904 ...  0.38760143  0.38760143
   0.38493794]
 [-0.32057241 -0.3303111  -0.32745469 ... -0.28540553 -0.28540553
  -0.28831995]
 ...
 [ 0.0101442   0.02869664  0.02258546 ...  0.29369506  0.29369506
   0.29816909]
 [-0.01548555  0.00307602 -0.00306675 ...  0.26241647  0.26241647
   0.26693276]
 [-0.21908485 -0.21991745 -0.21988698 ...  0.32269009  0.32269009
   0.32032618]]
CPU times: user 321 ms, sys: 73.1 ms, total: 394 ms
Wall time: 263 ms


In [47]:
vacancies_df.set_index('VacancyIdHashed').loc[relevant_vacancies].reset_index()

,VacancyIdHashed,City,SferaDeyatelnosti,GrafikRaboty,Professiya,Price,EmployerRating,Views,Contacts,Desrciption
0,1337100000117,Сочи,"Фитнес, салоны красоты",Сменный график,Массажист,45000.0,5.0,39,1,"Описание работодателя: работа в фитнес Центре,..."
1,1437174502155,Сочи,Продажи,Полный день,Менеджер по продажам,85000.0,5.0,95,1,Мы предлагаем для менеджера по продажам:\n\n🔥 ...
2,1438183500664,Сочи,Продажи,Неполный день,Комплектовщик,27500.0,5.0,240,19,В торговую компанию (Аптека) приглашается комп...
3,1434821500389,Сочи,"Фитнес, салоны красоты",Сменный график,Массажист,1.0,5.0,356,15,Описание работодателя:На постоянное место рабо...
4,1202028000565,Сочи,Продажи,Сменный график,Кассир,2250.0,NaN,319,17,Описание работодателя: \nТребования: \n🔹знание...
...,...,...,...,...,...,...,...,...,...,...
295,1055512250457,Сочи,Продажи,Сменный график,Продавец-консультант,30000.0,5.0,126,8,Ищем продавца-консультанта в самый большой и к...
296,1440021501659,Сочи,Продажи,Сменный график,Сборщик,42500.0,5.0,351,23,"В крупную сеть магазинов здорового питания ""Вк..."
297,1440021501659,Сочи,Продажи,Сменный график,Сборщик,42500.0,5.0,351,23,"В крупную сеть магазинов здорового питания ""Вк..."
298,893754000224,Сочи,Продажи,Сменный график,Продавец-консультант,50000.0,5.0,150,7,"Требуется продавец-консультант на двери,наполь..."


In [35]:
seekers_df = pd.read_csv('/kaggle/input/recsys-vacancies-seekers/subsample_seekers.csv')

In [32]:
seekers_df[seekers_df.SeekerIdHashed == 3493010241992275817]

,SeekerIdHashed,Gender,City,RegistrationTime,SferaDeyatelnosti,AllContactsForJob
196638,3493010241992275817,female,Сочи,2016-07-04 20:06:55.973613,Административная работа,1
196639,3493010241992275817,female,Сочи,2016-07-04 20:06:55.973613,"Банки, инвестиции",1
196640,3493010241992275817,female,Сочи,2016-07-04 20:06:55.973613,"Без опыта, студенты",7
196641,3493010241992275817,female,Сочи,2016-07-04 20:06:55.973613,Консультирование,4
196642,3493010241992275817,female,Сочи,2016-07-04 20:06:55.973613,Продажи,15
196643,3493010241992275817,female,Сочи,2016-07-04 20:06:55.973613,"Производство, сырьё, с/х",2
196644,3493010241992275817,female,Сочи,2016-07-04 20:06:55.973613,Строительство,1
196645,3493010241992275817,female,Сочи,2016-07-04 20:06:55.973613,"Туризм, рестораны",1


In [ ]:
seekers_df